<a href="https://colab.research.google.com/github/ShaheenPafiast/-simple-ai-python-projects/blob/main/FYP_SHOPLIFTING_Final_1_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Roboflow for dataset management
!pip install -q roboflow

# Install the necessary framework (YOLOv12 is often compatible with the Ultralytics ecosystem)
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 144.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.6 MB/s eta 0:00:00


In [2]:
from roboflow import Roboflow

# Initialize Roboflow
rf = Roboflow(api_key="5IfqtKaWrv15XPW7cbxl")
project = rf.workspace("abdullah-shaheen-i30o0").project("fyp-shoplift-byvkw")
version = project.version(1)

# Download for YOLOv12
dataset = version.download("yolov12")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to FYP-Shoplift-1 in yolov12:: 100%|██████████| 9452/9452 [00:01<00:00, 6108.22it/s]


In [3]:
from ultralytics import YOLO

# 1. Load the YOLOv12 model (assuming 'yolov12n.pt' for the nano version)
# If you have a specific .pt file for v12, use that path here
# The error indicates that 'yolov12' is not found. Ultralytics expects a model like 'yolov8n.pt'
# or a path to a custom .pt file. Let's try with a common YOLOv8 nano model.
# If you have a specific YOLOv12 model file, replace 'yolov8n.pt' with its path.
model = YOLO('yolov8n.pt')

# 2. Train the model
# 'data' points to the yaml file generated by the Roboflow download
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=50,      # Adjust epochs based on your needs
    imgsz=640,      # Standard image size
    plots=True      # Generates training graphs automatically
)



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/FYP-Shoplift-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, io

In [4]:
import os

# Assuming the Ultralytics results are in a 'runs' directory
runs_dir = 'runs/detect'

if os.path.exists(runs_dir):
    # Find the most recent training run directory
    train_dirs = [d for d in os.listdir(runs_dir) if os.path.isdir(os.path.join(runs_dir, d)) and d.startswith('train')]
    train_dirs.sort(key=lambda x: int(x.replace('train', '')) if x.replace('train', '').isdigit() else float('inf'), reverse=True)

    if train_dirs:
        latest_run_dir = os.path.join(runs_dir, train_dirs[0])
        print(f"Latest training results are saved in: {latest_run_dir}")
        print("Contents of the latest run directory:")
        for item in os.listdir(latest_run_dir):
            print(f"- {item}")
        print("\nLook for 'weights' folder (containing best.pt and last.pt) and various plots (.png) and results.csv here.")
    else:
        print(f"No training run directories found in {runs_dir}")
else:
    print(f"The directory '{runs_dir}' does not exist. Please ensure the model training has completed successfully.")

Latest training results are saved in: runs/detect/train
Contents of the latest run directory:
- val_batch0_labels.jpg
- train_batch8920.jpg
- val_batch2_pred.jpg
- train_batch8921.jpg
- train_batch1.jpg
- BoxR_curve.png
- train_batch2.jpg
- confusion_matrix.png
- val_batch0_pred.jpg
- labels.jpg
- val_batch1_pred.jpg
- BoxPR_curve.png
- val_batch1_labels.jpg
- train_batch0.jpg
- results.csv
- BoxP_curve.png
- args.yaml
- weights
- val_batch2_labels.jpg
- confusion_matrix_normalized.png
- train_batch8922.jpg
- BoxF1_curve.png
- results.png

Look for 'weights' folder (containing best.pt and last.pt) and various plots (.png) and results.csv here.


In [13]:
from google.colab import drive

In [15]:
import os
import shutil
import datetime

# Source directory (where Ultralytics saved its results)
runs_dir = 'runs/detect'

# Find the latest training run directory
train_dirs = [d for d in os.listdir(runs_dir) if os.path.isdir(os.path.join(runs_dir, d)) and d.startswith('train')]
train_dirs.sort(key=lambda x: int(x.replace('train', '')) if x.replace('train', '').isdigit() else float('inf'), reverse=True)

if train_dirs:
    latest_run_dir = os.path.join(runs_dir, train_dirs[0])

    # Destination directory in Google Drive
    base_destination_path = '/content/drive/MyDrive/YOLOv8_Training_Results'

    # Create a unique folder name using a timestamp
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    run_folder_name = f"{os.path.basename(latest_run_dir)}_{timestamp}"
    destination_path = os.path.join(base_destination_path, run_folder_name)

    # Create the base destination directory if it doesn't exist
    os.makedirs(base_destination_path, exist_ok=True)

    # Copy the entire directory
    print(f"Copying '{latest_run_dir}' to '{destination_path}'...")
    shutil.copytree(latest_run_dir, destination_path)
    print("Copy complete! Your results are now saved in Google Drive.")
    print(f"You can find your model weights (best.pt, last.pt) in: {destination_path}/weights")
else:
    print(f"No training run directories found in {runs_dir}. Please ensure the model training has completed successfully.")

Copying 'runs/detect/train' to '/content/drive/MyDrive/YOLOv8_Training_Results/train_20260122_144610'...
Copy complete! Your results are now saved in Google Drive.
You can find your model weights (best.pt, last.pt) in: /content/drive/MyDrive/YOLOv8_Training_Results/train_20260122_144610/weights


In [16]:
# Validate the model on the test set
metrics = model.val()

# Run inference on a test image from your dataset
results = model.predict(source=f"{dataset.location}/test/images", save=True)

print(f"Results saved to: {results[0].save_dir}")

Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1462.5±928.9 MB/s, size: 64.5 KB)
val: Scanning /content/FYP-Shoplift-1/valid/labels.cache... 758 images, 17 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 758/758 353.3Mit/s 0.0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 99, len(boxes) = 1058. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 48/48 6.1it/s 7.9s
                   all        758       1058      0.814      0.891      0.913      0.696
                normal        525        820       0.83      0.903      0.915       0.67
           shoplifting        238        238      0.797      0.878      0.911      0.722
Speed: 1.2ms preprocess, 2.8

In [32]:
# Install Gradio library for creating web interfaces
!pip install -q gradio

In [33]:
import gradio as gr
from ultralytics import YOLO
import os
import cv2
import numpy as np

# --- 1. Load the Trained Model ---
# This reuses the logic to find your best trained model
runs_dir = 'runs/detect'
train_dirs = [d for d in os.listdir(runs_dir) if os.path.isdir(os.path.join(runs_dir, d)) and d.startswith('train')]
train_dirs.sort(key=lambda x: int(x.replace('train', '')) if x.replace('train', '').isdigit() else float('inf'), reverse=True)

model = None
if train_dirs:
    latest_run_dir = os.path.join(runs_dir, train_dirs[0])
    model_path = os.path.join(latest_run_dir, 'weights', 'best.pt')
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = YOLO(model_path)
    else:
        print(f"Error: Model weights not found at {model_path}")
else:
    print("No trained model found. Please train the model first or provide the path to best.pt")

# Ensure model is loaded before proceeding
if model is None:
    raise ValueError("YOLOv8 model could not be loaded. Please ensure training completed successfully or the path is correct.")

# --- 2. Define the Inference Function for Gradio ---
def predict_image(image: np.ndarray) -> np.ndarray:
    """
    Performs object detection on the input image using the loaded YOLO model.
    Returns the image with bounding boxes and labels drawn on it.
    """
    if model is None:
        return image # Return original image if model not loaded

    # Convert Gradio's image input (numpy array) to a format YOLO can process
    # YOLO's predict method can directly take a numpy array or a PIL Image
    results = model.predict(source=image, conf=0.25) # conf is confidence threshold

    # Process results and draw on the image
    # Ultralytics results objects often contain the annotated image directly
    if results:
        # The .plot() method returns a numpy array of the image with detections drawn
        annotated_image = results[0].plot()
        return annotated_image
    else:
        return image # Return original image if no detections

# --- 3. Create and Launch Gradio Interface ---
iface = gr.Interface(fn=predict_image,
                     inputs=gr.Image(type="numpy", label="Upload Image"),
                     outputs=gr.Image(type="numpy", label="Predicted Image"),
                     title="Shoplifting Detection with YOLOv8",
                     description="Upload an image to detect instances of 'normal' and 'shoplifting' behavior.")

# You can choose to queue to handle multiple requests, and debug=True for more verbose output
iface.launch(debug=True, share=True)


Loading model from: runs/detect/train/weights/best.pt
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d331c86e4293670174.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 448x640 2 shopliftings, 38.6ms
Speed: 2.4ms preprocess, 38.6ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)
Keyboard interruption in main thread... closing server.


KeyboardInterrupt: 

In [34]:
import gradio as gr
from ultralytics import YOLO
import os

# --- 1. Load the Trained Model ---
# This reuses the logic to find your best trained model
runs_dir = 'runs/detect'
train_dirs = [d for d in os.listdir(runs_dir) if os.path.isdir(os.path.join(runs_dir, d)) and d.startswith('train')]
train_dirs.sort(key=lambda x: int(x.replace('train', '')) if x.replace('train', '').isdigit() else float('inf'), reverse=True)

model = None
if train_dirs:
    latest_run_dir = os.path.join(runs_dir, train_dirs[0])
    model_path = os.path.join(latest_run_dir, 'weights', 'best.pt')
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = YOLO(model_path)
    else:
        print(f"Error: Model weights not found at {model_path}")
else:
    print("No trained model found. Please train the model first or provide the path to best.pt")

# Ensure model is loaded before proceeding
if model is None:
    raise ValueError("YOLOv8 model could not be loaded. Please ensure training completed successfully or the path is correct.")

# --- 2. Define the Inference Function for Video ---
def predict_video(video_path: str) -> str:
    """
    Performs object detection on the input video using the loaded YOLO model.
    Returns the path to the annotated video.
    """
    if model is None:
        return "Error: Model not loaded." # Return error message if model not loaded

    if not video_path or not os.path.exists(video_path):
        return "Error: Please upload a valid video file."

    print(f"Running inference on video: {video_path}")
    # YOLO's predict method can directly take a video path
    results = model.predict(source=video_path, conf=0.25, save=True) # conf is confidence threshold

    # The predicted video will be saved in a new directory inside 'runs/detect/predict'
    # The directory name will be something like 'predict' or 'predict2', etc.
    # We need to find the path to the saved video.
    if results:
        output_dir = results[0].save_dir
        # Assuming the output video has the same base name as the input video
        output_video_name = os.path.basename(video_path)
        output_video_path = os.path.join(output_dir, output_video_name)
        print(f"Annotated video saved to: {output_video_path}")
        return output_video_path
    else:
        return video_path # Return original video path if no detections or issue

# --- 3. Create and Launch Gradio Interface for Video ---
iface_video = gr.Interface(fn=predict_video,
                         inputs=gr.Video(label="Upload Video"),
                         outputs=gr.Video(label="Predicted Video"),
                         title="Shoplifting Detection with YOLOv8 (Video)",
                         description="Upload a video to detect instances of 'normal' and 'shoplifting' behavior.")

# You can choose to queue to handle multiple requests, and debug=True for more verbose output
iface_video.launch(debug=True, share=True)

Loading model from: runs/detect/train/weights/best.pt
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6eb71a6ee0fd39c397.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error

Running inference on video: /tmp/gradio/e8011d71e2c97c8f7273bd6099a489026d2cf2065ff0088b52e728254b590922/vid1shoplift.webm

WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/223) /tmp/gradio/e8011d71e2c97c8f7273bd6099a489026d2cf2065ff0088b52e728254b590922/vid1shoplift.webm: 384x640 1 shoplifting, 35.5ms
video 1/1 (frame 2/223) /tmp/gradio/e8011d71e2c97c8f7273bd6099a489026d2cf2065ff0088b52e728254b590922/vid1shoplift.webm: 384x640 1 shoplifting, 5.6ms
video 1/1 (frame 3/223) /tmp/gradio/

In [19]:
!unzip /content/pafiast_shoplifting_dataset.yolov8.zip

Archive:  /content/pafiast_shoplifting_dataset.yolov8.zip
  inflating: README.dataset.txt      
  inflating: README.roboflow.txt     
  inflating: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/Shoplifting001_x264_15_mp4-20_jpg.rf.5a83808833d7d208ba7807d6e09e6a90.jpg  
 extracting: test/images/WhatsApp-Video-2023-10-22-at-12_10_42-AM-online-video-cutter_com-_mp4-49_jpg.rf.68af0ab17c5b8273d688e7b5e91fa4dc.jpg  
 extracting: test/images/aahil2_mp4-2_jpg.rf.520c57535b089981390398906dc54891.jpg  
 extracting: test/images/beer-can-theft_mp4-14_jpg.rf.bd3578beef3500d3a1c79b37a122a4bd.jpg  
   creating: test/labels/
 extracting: test/labels/Shoplifting001_x264_15_mp4-20_jpg.rf.5a83808833d7d208ba7807d6e09e6a90.txt  
 extracting: test/labels/WhatsApp-Video-2023-10-22-at-12_10_42-AM-online-video-cutter_com-_mp4-49_jpg.rf.68af0ab17c5b8273d688e7b5e91fa4dc.txt  
 extracting: test/labels/aahil2_mp4-2_jpg.rf.520c57535b089981390398906dc54891.txt  
 extra

In [21]:
from ultralytics import YOLO
import os

# --- Find the latest trained model ---
runs_dir = 'runs/detect'
train_dirs = [d for d in os.listdir(runs_dir) if os.path.isdir(os.path.join(runs_dir, d)) and d.startswith('train')]
train_dirs.sort(key=lambda x: int(x.replace('train', '')) if x.replace('train', '').isdigit() else float('inf'), reverse=True)

model = None
if train_dirs:
    latest_run_dir = os.path.join(runs_dir, train_dirs[0])
    model_path = os.path.join(latest_run_dir, 'weights', 'best.pt')
    print(f"Loading model from: {model_path}")
    model = YOLO(model_path)
else:
    print("No trained model found. Please train the model first.")

if model:
    # 1. Validate the model on the test set
    # Note: Ultralytics' val() method typically runs on the 'val' split defined in data.yaml.
    # If your data.yaml has a 'test' split, you can specify `split='test'`.
    # Assuming 'dataset' is available from Roboflow download, its 'test/images' would be the actual test set.
    print("\n--- Validating Model ---")
    metrics = model.val()  # No need to pass data path explicitly if defined in data.yaml

    # Display key metrics
    print(f"\nmAP50: {metrics.box.map50}")
    print(f"mAP50-95: {metrics.box.map}")
    print(f"Classes: {metrics.names}")

    # 2. Run inference on test images and save the results
    # Assuming `dataset` object is available from Roboflow download for its location
    if 'dataset' in locals():
        test_images_path = os.path.join(dataset.location, 'test', 'images')
        if os.path.exists(test_images_path):
            print(f"\n--- Running Inference on Test Images from {test_images_path} ---")
            results = model.predict(source=test_images_path, save=True, conf=0.25) # conf is confidence threshold
            print(f"\nInference results (images with detections) saved to: {results[0].save_dir}")
            print("You can view these images in the specified directory.")
        else:
            print(f"Error: Test images directory not found at {test_images_path}.")
    else:
        print("Error: The 'dataset' variable is not defined. Please ensure the Roboflow dataset download cell has been executed.")
else:
    print("Model not loaded. Cannot perform validation or inference.")


Loading model from: runs/detect/train/weights/best.pt

--- Validating Model ---
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 73 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1315.9±710.4 MB/s, size: 63.7 KB)
val: Scanning /content/valid/labels... 95 images, 95 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 95/95 2.9Kit/s 0.0s
val: New cache created: /content/valid/labels.cache
WARNING ⚠️ Labels are missing or empty in /content/valid/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 2.4it/s 2.5s


Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in divide
Mean of empty slice.
invalid value encountered in divide
Mean of empty slice.
invalid value encountered in divide
Mean of empty slice.


                   all         95          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels
Speed: 5.9ms preprocess, 5.0ms inference, 0.0ms loss, 4.3ms postprocess per image
Results saved to /content/runs/detect/val3

mAP50: 0.0
mAP50-95: 0.0
Classes: {0: 'normal', 1: 'shoplifting'}

--- Running Inference on Test Images from /content/FYP-Shoplift-1/test/images ---

image 1/395 /content/FYP-Shoplift-1/test/images/1_mp4-12_jpg.rf.be4a96ab836f466f6a9288a035ede5a2.jpg: 352x640 1 normal, 6.6ms
image 2/395 /content/FYP-Shoplift-1/test/images/21714680-21714680-1-_jpg.rf.0ca35415a7c27995e08e27aa9c15bb5c.jpg: 480x640 1 normal, 6.7ms
image 3/395 /content/FYP-Shoplift-1/test/images/2346-summer-steal-candy_mp4-20_jpg.rf.ae466116f918b86d976ad429a2167584.jpg: 512x640 1 normal, 6.6ms
image 4/395 /content/FYP-Shoplift-1/test/images/2346-summer-steal-candy_mp4-31_jpg.rf.bb6e453800792018af5fb641a2612a09.jpg: 512x640 2 normals, 

#### Mount Google Drive (if running in Colab)

In [22]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: Mountpoint must not already contain files

#### Load the Trained Model from Google Drive

Replace the `model_path_in_drive` with the actual path to your `best.pt` file in your Google Drive. This path corresponds to where the notebook previously saved your model.

In [23]:
from ultralytics import YOLO
import os

# --- IMPORTANT: Update this path to your saved model in Google Drive ---
# The timestamped folder is unique to your run.
# For example: '/content/drive/MyDrive/YOLOv8_Training_Results/train_20260122_144610/weights/best.pt'
model_path_in_drive = '/content/drive/MyDrive/YOLOv8_Training_Results/train_20260122_144610/weights/best.pt'

if os.path.exists(model_path_in_drive):
    print(f"Loading trained model from: {model_path_in_drive}")
    model = YOLO(model_path_in_drive)
else:
    print(f"Error: Model not found at {model_path_in_drive}. Please check the path.")
    model = None

Loading trained model from: /content/drive/MyDrive/YOLOv8_Training_Results/train_20260122_144610/weights/best.pt


#### Run Inference on New Images

You can upload new images to your Colab session or specify a path to images already in your Google Drive. Make sure to update the `input_source` variable below.

In [25]:
if model:
    # --- IMPORTANT: Update this variable with your input images or video ---
    # Examples:
    # 1. To process a single image you uploaded: 'my_new_image.jpg'
    # 2. To process all images in a folder you uploaded: 'new_images_folder/' (after uploading the folder to Colab)
    # 3. To process a video you uploaded: 'my_new_video.mp4'
    # 4. To process images directly from Google Drive: '/content/drive/MyDrive/my_test_images_folder/'
    # 5. To process the test images that came with your Roboflow dataset (if 'dataset' variable is still defined):
    #    input_source = os.path.join(dataset.location, 'test', 'images')

    input_source = 'path/to/your/input_images_or_video' # <--- UPDATE THIS LINE

    if os.path.exists(input_source):
        print(f"Running inference on: {input_source}")
        # You can adjust the confidence threshold (conf) as needed
        results = model.predict(source=input_source, save=True, conf=0.25)
        print(f"\nInference results saved to: {results[0].save_dir}")
        print("You can find the images/videos with predictions in this directory.")
    else:
        print(f"Error: Input source not found at {input_source}. Please ensure the file/directory exists and the path is correct.")
else:
    print("Model not loaded. Cannot perform inference.")

Error: Input source not found at path/to/your/input_images_or_video. Please ensure the file/directory exists and the path is correct.


In [20]:
import os
from ultralytics import YOLO

# --- Find the latest trained model ---
runs_dir = 'runs/detect'
train_dirs = [d for d in os.listdir(runs_dir) if os.path.isdir(os.path.join(runs_dir, d)) and d.startswith('train')]
train_dirs.sort(key=lambda x: int(x.replace('train', '')) if x.replace('train', '').isdigit() else float('inf'), reverse=True)

if train_dirs:
    latest_run_dir = os.path.join(runs_dir, train_dirs[0])
    model_path = os.path.join(latest_run_dir, 'weights', 'best.pt')
    print(f"Loading model from: {model_path}")
    model = YOLO(model_path)
else:
    print("No trained model found. Please train the model first.")
    model = None # Ensure model is None if not found

# --- Perform inference on a video ---
if model:
    # Replace 'path/to/your/video.mp4' with the actual path to your video file
    # Example: If you upload a video named 'my_test_video.mp4' to your Colab environment,
    # you would use 'my_test_video.mp4'
    video_path = 'path/to/your/video.mp4' # <--- IMPORTANT: UPDATE THIS PATH

    if os.path.exists(video_path):
        print(f"Running inference on video: {video_path}")
        results = model.predict(source=video_path, save=True, conf=0.25) # conf is confidence threshold
        print(f"Video inference results saved to: {results[0].save_dir}")
    else:
        print(f"Error: Video file not found at {video_path}. Please update the 'video_path' variable and ensure the file exists.")


Loading model from: runs/detect/train/weights/best.pt
Error: Video file not found at path/to/your/video.mp4. Please update the 'video_path' variable and ensure the file exists.


First, let's confirm the path to your test images from the downloaded Roboflow dataset.

In [31]:
import os

# The dataset object from Roboflow stores the location of the downloaded and extracted dataset.
# This should contain the test images.

if 'dataset' in locals():
    test_images_path = os.path.join(dataset.location, 'test', 'images')
    print(f"The path to your test images is: {test_images_path}")

    if os.path.exists(test_images_path):
        print(f"Found {len(os.listdir(test_images_path))} images in the test directory.")
        print("The first 5 image filenames are:")
        for i, filename in enumerate(os.listdir(test_images_path)[:5]):
            print(f"- {filename}")
    else:
        print(f"Error: Test image directory not found at {test_images_path}.")
else:
    print("Error: The 'dataset' variable is not defined. Please ensure the Roboflow dataset download cell has been executed.")

The path to your test images is: /content/FYP-Shoplift-1/test/images
Found 395 images in the test directory.
The first 5 image filenames are:
- aahil3_mp4-18_jpg.rf.8cc23c9cc828c4cfa3041de4d3ec7ec2.jpg
- output_mp4-1617_jpg.rf.18811917ea218b7325d9dbda617449dd.jpg
- frame_000048_jpg.rf.9fc41d1ef1300223619e05cdbbbb9e28.jpg
- Shoplifting001_x264_12_mp4-26_jpg.rf.8b7fc15184cf59abfe79bd7e458d0a47.jpg
- Furti-in-negozio_-tutta-l-oscenita-del-furto-in-15-secondi-_out0032_jpg.rf.28533aed8383bfc407b9223e23a9801d.jpg


Now, to run inference on all these images using your trained model, you can use the `model.predict()` function directly on this directory. The code in cell `rY_Tz1Iw1271` already does this, but here's a reminder of how it works: